### Import Libraries

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
# renderer for jupyterlab
pio.renderers.default='jupyterlab'
# renderer for vs code
# pio.renderers.default='vscode'
# renderer for colab
# pio.renderers.default='colab'
pio.templates.default = "plotly_dark"
import numpy as np
import statsmodels.api as sm
import datetime
from plotly.subplots import make_subplots
from ipywidgets import widgets,Layout
import warnings
warnings.simplefilter(action='ignore')

### Importing the datasets

In [2]:
rain=pd.read_csv(r'../data/rainfall-by-region.csv')
regions=list(rain.columns.values)[5:]
years=list(rain['year'].unique())
monsoon_list=[i for i in range(150,281)]
rain_monsoon=rain[rain['day-number'].isin(monsoon_list)]

In [3]:
rain=rain.set_index('date',drop=False)

### Plotting Function

In [4]:
# dropdown widget
region = widgets.Dropdown(
    options=regions,
    value=regions[0], description='Region',layout=Layout(margin='20px 20px 20px 20px'))
container=widgets.HBox([region])

In [5]:
def initial_plot(x_axis,df,iterate_list,title,x_axis_title):
#     list_type=year
    traces=[]
    if len(iterate_list)==0:
        traces.append(go.Scatter(x=df[x_axis],y=df[regions[0]],name=regions[0]))
    else:
        for i in iterate_list:
            temp = df[df['year'] == i]
            traces.append(go.Scatter(x=temp[x_axis],y=temp[regions[0]],name=str(i)))
    layout = go.Layout(title=dict(text=title),xaxis=dict(title=x_axis_title,rangeslider=dict(visible=True)),yaxis=dict(title='Rainfall (mm)'))
    return go.FigureWidget(data=traces, layout=layout)

In [6]:
def plot_fig(id_r,title,x_axis,g,data,iterate_list):
    var={i.split()[0].lower():i for i  in regions}
    id_region=var[id_r]
    with g.batch_update():
        if len(iterate_list)!=0:
            for i in iterate_list:
                df = data[data['year'] == i]
                g.data[i-1957].y=df[id_region]
                g.data[i-1957].name=str(i)
                g.data[i-1957].x=df[x_axis]
        else:
            g.data[0].y=data[id_region]
            g.data[0].name=str(title)
            g.data[0].x=data[x_axis]

In [7]:
def response(change):
    val=region.value.split()[0].lower()
    plot_fig(val,region.value,'day-number',g,rain_monsoon,years)
    plot_fig(val,region.value,'month',g_mean_monthly_year,mean_monthly_year,years)
    plot_fig(val,region.value,'month',g_median_monthly_year,median_monthly_year,years)
    plot_fig(val,region.value,'year',g_mean_yearly,mean_yearly,[])
    plot_fig(val,region.value,'year',g_median_yearly,median_yearly,[])
    plot_fig(val,region.value,'day-number',g_mean_daily,mean_daily,[])
    plot_fig(val,region.value,'day-number',g_median_daily,median_daily,[])
    plot_fig(val,region.value,'day-number',g_mean_daily_monsoon,mean_daily_monsoon,[])
    plot_fig(val,region.value,'day-number',g_median_daily_monsoon,median_daily_monsoon,[])

In [8]:
region.observe(response,names='value')

In [9]:
rain.fillna(0,inplace=True)

In [10]:
rain.isnull().sum()

date              0
year              0
day-number        0
date.1            0
month             0
day-of-month      0
ajmer             0
alwar             0
banswara          0
baran             0
barmer            0
bharatpur         0
bhilwara          0
bikaner           0
bundi             0
chittorgarh       0
churu             0
dausa             0
dholpur           0
dungarpur         0
hanumangarh       0
jaipur            0
jaisalmer         0
jalore            0
jhunjhunu         0
jodhpur           0
karoli            0
kota              0
nagaur            0
pali              0
pratapgarh        0
rajsamand         0
sawai madhopur    0
sikar             0
sirohi            0
ganganagar        0
tonk              0
udaipur           0
aw                0
bshw              0
cwq               0
bwhw              0
sub humid         0
humid             0
very humid        0
semi arid         0
arid              0
daw               0
caw               0
ead               0


In [11]:
g=initial_plot('day-number',rain_monsoon,years,'Rain observed on daily basis (monsoon time)','Day of year')
total_container=widgets.VBox([container,g])
display(total_container)

### Monthly Basis Classification

In [12]:
mean_monthly_year=rain.groupby(['year','month'],as_index=False)[regions].mean()
median_monthly_year=rain.groupby(['year','month'],as_index=False)[regions].median()

In [13]:
g_mean_monthly_year=initial_plot('month',mean_monthly_year,years,'Mean rain observed on monthly basis','Month')
g_median_monthly_year=initial_plot('month',median_monthly_year,years,'Median rain observed on monthly basis','Month')
total_container=widgets.VBox([container,g_mean_monthly_year,g_median_monthly_year])
display(total_container)

### Yearly Basis Classification

In [14]:
mean_yearly=rain.groupby(['year'],as_index=False)[regions].mean()
median_yearly=rain.groupby(['year'],as_index=False)[regions].median()

In [15]:
g_mean_yearly=initial_plot('year',mean_yearly,[],'Mean rain observed on yearly basis','Year')
g_median_yearly=initial_plot('year',median_yearly,[],'Median rain observed on yearly basis','Year')
display(region,g_mean_yearly,g_median_yearly)

Dropdown(description='Region', layout=Layout(margin='20px 20px 20px 20px'), options=('day-of-month', 'ajmer', …

FigureWidget({
    'data': [{'name': 'day-of-month',
              'type': 'scatter',
              'uid': '9b…

FigureWidget({
    'data': [{'name': 'day-of-month',
              'type': 'scatter',
              'uid': '1f…

### Daily Basis Classification

In [16]:
mean_daily=rain.groupby(['day-number'],as_index=False)[regions].mean()
median_daily=rain.groupby(['day-number'],as_index=False)[regions].median()

In [17]:
g_mean_daily=initial_plot('day-number',mean_daily,[],'Mean distribution of Rainfall','Day of year')
g_median_daily=initial_plot('day-number',median_daily,[],'Median distribution of Rainfall','Day of year')
display(region,g_mean_daily,g_median_daily)

Dropdown(description='Region', layout=Layout(margin='20px 20px 20px 20px'), options=('day-of-month', 'ajmer', …

FigureWidget({
    'data': [{'name': 'day-of-month',
              'type': 'scatter',
              'uid': '63…

FigureWidget({
    'data': [{'name': 'day-of-month',
              'type': 'scatter',
              'uid': 'ce…

In [18]:
mean_daily_monsoon=rain_monsoon.groupby(['day-number'],as_index=False)[regions].mean()
median_daily_monsoon=rain_monsoon.groupby(['day-number'],as_index=False)[regions].median()

In [19]:
g_mean_daily_monsoon=initial_plot('day-number',mean_daily_monsoon,[],'Mean distribution of Rainfall (monsoon time)','Day of year')
g_median_daily_monsoon=initial_plot('day-number',median_daily_monsoon,[],'Median distribution of Rainfall (monsoon time)','Day of year')
display(region,g_mean_daily_monsoon,g_median_daily_monsoon)

Dropdown(description='Region', layout=Layout(margin='20px 20px 20px 20px'), options=('day-of-month', 'ajmer', …

FigureWidget({
    'data': [{'name': 'day-of-month',
              'type': 'scatter',
              'uid': '63…

FigureWidget({
    'data': [{'name': 'day-of-month',
              'type': 'scatter',
              'uid': '07…